<a href="https://colab.research.google.com/github/Aniroodh1234/Image_Segmentation/blob/main/Image_segmentation_KMeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧰 Install necessary libraries
!pip install opencv-python matplotlib numpy


In [8]:
# 📦 Import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# 📂 Define image directory
image_folder = "Image"  # Folder where your images are stored

# 🖼️ List all image files
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))]

# 🔍 Display all images
for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(5,5))
    plt.imshow(img_rgb)
    plt.title(f"Original Image: {img_file}")
    plt.axis('off')
    plt.show()


In [10]:
# 🎨 Function to perform K-Means segmentation
def kmeans_segmentation(image_path, K=3):
    # Step 1: Read and convert image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Step 2: Reshape image into 2D array of pixels
    pixel_values = img_rgb.reshape((-1, 3))
    pixel_values = np.float32(pixel_values)

    # Step 3: Define criteria and apply KMeans
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(
        pixel_values, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
    )

    # Step 4: Convert centers back to uint8 (RGB colors)
    centers = np.uint8(centers)

    # Step 5: Replace pixel values with cluster center
    segmented_data = centers[labels.flatten()]

    # Step 6: Reshape back to original image dimensions
    segmented_image = segmented_data.reshape(img_rgb.shape)

    return img_rgb, segmented_image


In [ ]:
# 🎯 Apply segmentation to all images in folder
K = 4  # Number of clusters (change this to experiment)

for img_file in image_files:
    img_path = os.path.join(image_folder, img_file)
    original, segmented = kmeans_segmentation(img_path, K)

    # Display results side-by-side
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(original)
    plt.title("Original Image")
    plt.axis('off')

    plt.subplot(1,2,2)
    plt.imshow(segmented)
    plt.title(f"K-Means Segmentation (K={K})")
    plt.axis('off')

    plt.suptitle(f"Segmentation Result for {img_file}", fontsize=14)
    plt.show()
